# ConsumerProducts Vs Toys - InceptionV3

**A Module with hyperparameter tuning included without transfer learning,trial**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
os.listdir('../input')

['consumer-products-vs-toys']

In [3]:
from tensorflow.keras.layers import Input , Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline


from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

import tensorflow as tf 
from keras.models import load_model
from keras import optimizers, losses, activations, models

In [4]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
img_size = [150,150]
train_path = '/kaggle/input/consumer-products-vs-toys/Data/train'
test_path = '/kaggle/input/consumer-products-vs-toys/Data/test'
#vgg = VGG16(input_shape=img_size+[3] , weights='imagenet',include_top= False)
#vgg.layers
new_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150, 150,3))

87916544/87910968 [==============================] - 1s 0us/step


In [7]:
'''new_model.summary()'''

'new_model.summary()'

In [8]:
for layer in new_model.layers:
    layer.trainable = False

In [9]:
'''new_model.summary()'''

'new_model.summary()'

In [10]:
#Adding ending layers
#x1 = Dropout(0.3)(vgg.output)
x = Flatten()(new_model.output)#Final layer
#x2 = Dropout(0.2)(x)
#x3 = Flatten()(x2)#Final layer
x = Dense(1024,activation='relu')(x)
x = Dropout(0.2)(x)
prediction = Dense(1,activation='sigmoid')(x)
model = Model(inputs=new_model.input,outputs=prediction)

In [11]:
'''model.summary()'''

'model.summary()'

In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [13]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
from keras.preprocessing.image import ImageDataGenerator
training_set = train_datagen.flow_from_directory('/kaggle/input/consumer-products-vs-toys/Data/train',
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary')
test_set = test_datagen.flow_from_directory('/kaggle/input/consumer-products-vs-toys/Data/test',
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary')


Found 1730 images belonging to 2 classes.
Found 742 images belonging to 2 classes.


In [16]:
import tensorflow as tf
class MyCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if(logs.get('accuracy')>0.955 and logs.get('val_accuracy')>0.915):
                print("\nReached n% accuracy so cancelling training!")
                self.model.stop_training = True

In [17]:
callbacks=MyCallback()

In [18]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')



In [ ]:
fit_  = model.fit(training_set,
                           validation_data=test_set,
                           epochs=25,
                           steps_per_epoch=len(training_set),
                           validation_steps=len(test_set),verbose=1,
                  callbacks = [callbacks,checkpoint,early])

Epoch 1/25
55/55 [==============================] - 20s 364ms/step - loss: 3.2930 - accuracy: 0.7890 - val_loss: 1.0351 - val_accuracy: 0.7116
Epoch 2/25
55/55 [==============================] - 15s 275ms/step - loss: 0.5335 - accuracy: 0.8353 - val_loss: 0.4716 - val_accuracy: 0.8032
Epoch 3/25
55/55 [==============================] - 15s 266ms/step - loss: 0.2871 - accuracy: 0.8855 - val_loss: 0.4757 - val_accuracy: 0.8208
Epoch 4/25
55/55 [==============================] - 16s 295ms/step - loss: 0.2321 - accuracy: 0.8971 - val_loss: 0.4242 - val_accuracy: 0.8167
Epoch 5/25
55/55 [==============================] - 14s 262ms/step - loss: 0.1897 - accuracy: 0.9225 - val_loss: 0.4183 - val_accuracy: 0.8315
Epoch 6/25
55/55 [==============================] - 16s 284ms/step - loss: 0.1892 - accuracy: 0.9133 - val_loss: 0.4788 - val_accuracy: 0.8181
Epoch 7/25
48/55 [=========================>....] - ETA: 1s - loss: 0.1765 - accuracy: 0.9356

**Continuing Training**

In [ ]:
fit_ = model.fit(training_set,
                           validation_data=test_set,
                           epochs=300,
                           steps_per_epoch=len(training_set),
                           validation_steps=len(test_set),verbose=1,
                  callbacks = [callbacks,checkpoint,early],initial_epoch=25)

In [ ]:
'''model.save("final_model.h5")'''

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(fit_.history['accuracy'])
plt.plot(fit_.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'],loc='upper left')
plt.show()


In [ ]:
plt.plot(fit_.history['loss'])
plt.plot(fit_.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(['train','val'],loc='upper left')
plt.show()
